In [1]:
from kafka.consumer import KafkaConsumer    #to initialize the consumer
from kafka.producer.kafka import KafkaProducer    #to initialize the producer
from kafka.errors import KafkaError
from struct import pack, unpack
import mmh3     #for hashing
import math     
import json    #for loading data

In [2]:
#to use kafka admin ===> needed for topic creation
from kafka.admin import KafkaAdminClient, NewTopic
import re
import string

In [3]:
#register kafka adminclient
admin_client = KafkaAdminClient(bootstrap_servers='127.0.0.1:9092')

In [5]:
#Generate 20 topics and save its value in a list called venueTopic_list
venueTopic_list =[]
for i in range(0,20):
    s = 'venueTopic' + str(i)   #create topic name
    venueTopic_list.append(s)   #add topic to list
    new_topics = NewTopic(s, num_partitions=1, replication_factor=1)    #mention details for topic
    admin_client.create_topics([new_topics])     #create new topic

In [2]:
#bin\windows\kafka-console-producer --bootstrap-server 127.0.0.1:9092 --topic input-topic
#bin\windows\kafka-console-consumer --bootstrap-server 127.0.0.1:9092 --topic output-topic

In [6]:
import pickle  
with open('assignment3_Q1_pickle', 'rb') as f:
   res = pickle.load(f)               # load method is used to load the contents of a pickle file


In [7]:
#used to determine trailing number of zeros
def trailing_zeros(n):
    if n == 0:
        return 16   #if number is zero we return 16 as max number
    
    p = 0    #stores trailing zeros
    while (n >> p) & 1 == 0:   #else right shift the number and check if the last bit is zero or not
        p += 1     #if its zero then increment the count
    
    return p    #return p

In [8]:
#performs flajolet martin process
def flajolet_martin(hashes):
    maxZero = 0   #stroes max trailing zeros
    #for every has find which hash has max traling zeros
    for i in hashes:   
        maxZero = max(maxZero, trailing_zeros(i))
    return 2 ** maxZero   #return 2^(max trailing zeros)

In [23]:
#We divided all the venues into 20 buckets
key = len(list(res.keys()))
bucket = key/20

#initlaize the producer
producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])

i=0
j=0
topic = venueTopic_list[j]  #for first run

#initialize consumer with first topic
consumer = KafkaConsumer(topic, bootstrap_servers=['127.0.0.1:9092'], auto_offset_reset='earliest',
                         enable_auto_commit=True,
                         group_id='flajolet-martin')

#For data in every bucket we initialize the consumer to that particular topic and send data via producer. 
#For evry new bucket change the topic and place producer and consumer on that topic.
#We apply the same steps and output is directed to output topic.

for k,v in res.items():
    i = i+1
    if i%bucket == 0:
        j = j+1
        if j ==20:
            break
        topic = venueTopic_list[j]
        consumer.close()
        consumer = KafkaConsumer(topic, bootstrap_servers=['127.0.0.1:9092'], auto_offset_reset='earliest',
                         enable_auto_commit=True,
                         group_id='flajolet-martin')
        
    producer.send(topic, str(v).encode('utf-8'))
    message = next(consumer)
    value = message.value
    words = value.decode('utf-8').split(',')
    hashes = [mmh3.hash(word) for word in words]
    estimate = flajolet_martin(hashes)
    producer.send('output-topic', (str(k) +"   ===>   " + str(estimate)).encode('utf-8'))
                  
                  


In [16]:
#initialize kafka producer
producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])

#Initialize kafka consumer
consumer = KafkaConsumer('myTopic05', bootstrap_servers=['127.0.0.1:9092'], auto_offset_reset='earliest',
                         enable_auto_commit=True,
                         group_id='flajolet-martin')


for k,v in res.items():    #for all items in dataset

    producer.send('myTopic05', str(v).encode('utf-8')) #send the author data via producer
    
    message = next(consumer)  #read it on consumer
    value = message.value #extract value from it
    words = value.decode('utf-8').split(',') #sdecode the data and plit the data via comma. Store it as list 
    hashes =[]   #used to store all the hashes
    for word in words:  #store the hashed data into hash array
        hashes.append(mmh3.hash(word)) 
        
    estimate = flajolet_martin(hashes)   #run flajolet martin and obtain the unique elements
    
    #send the data via producer on target consumer
    producer.send('output-topic', (str(k) +"   ===>   " + str(estimate)).encode('utf-8'))

In [11]:
words

['D Haussler',
 'C L Monma',
 'V K Wei',
 'N Robertson',
 'P Seymour',
 'N Robertson',
 'P Seymour',
 'Gyula Katona',
 'N Sauer',
 'A Mukhopadhyay',
 'S Smorodinsky',
 'Y Yuditsky',
 'J H Van Lint',
 'N Alon',
 'G Ding',
 'B Oporowski',
 'D Vertigan',
 'M Molloy',
 'Joseph B Kruskal',
 'F Grande',
 'R Sanyal',
 'H Walther',
 'W O Alltop',
 'E F Assmus',
 'H F Mattson',
 'L H Haines',
 'H Leonard',
 ' Haines',
 'M D Plummer']